In [0]:
import pandas as pd
import delta


def table_exists(catalog, database, table):
  count = (spark.sql(f"SHOW TABLES FROM {catalog}.{database}")
           .filter(f"database = '{database}' AND tableName = '{table}' ")
           .count())
  
  return count == 1

In [0]:
catalog = "bronze"
schema = "f1_world"
tableName = dbutils.widgets.get("tableName")
TARGET_RACEID = 1141
files_split = ["constructor_results" ,"constructor_standings", "driver standings", "lap_times", "pit_stops", "qualifying", "results", "sprint_results"]

In [0]:
if not table_exists(catalog, schema, tableName):
  print(f"Criando tabela {tableName}")

  if tableName in files_split:
    df_split = spark.read.format("csv").options(header="true").load(f"/Volumes/raw/{schema}/{tableName}/")
    

    df_batch = df_split.filter(df_split.raceId != TARGET_RACEID)
    df_batch.coalesce(1).write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{tableName}")
    print("Tabela delta criado com sucesso")
    

    df_streaming = df_split.filter(df_split.raceId == TARGET_RACEID)
    df_streaming_pandas = df_streaming.toPandas()
    df_streaming_pandas.to_parquet(f"/Volumes/raw/{schema}/streaming_source/{tableName}.parquet", index=False)
    print("Arquivo streaming criado com sucesso")

  else:
    df_full = spark.read.format("csv").options(header="true").load(f"/Volumes/raw/{schema}/{tableName}/")

    df_full.coalesce(1).write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{tableName}")
else:
  print(f"Tabela {tableName} ja existe")
                                        
    